# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [32]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [33]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.34,87,92,15.35,PN,1691531783
1,1,selfoss,63.9331,-20.9971,12.25,83,79,0.76,IS,1691531783
2,2,ushuaia,-54.8000,-68.3000,3.81,75,40,1.03,AR,1691531783
3,3,tazovsky,67.4667,78.7000,8.76,97,100,4.32,RU,1691531673
4,4,avarua,-21.2078,-159.7750,23.03,60,75,6.17,CK,1691531783


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [39]:
%%capture --no-display

# Configure the map plot
humidity_map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height=500,
    size = "Humidity",
    color = "City",
    alpha = 0.8)

# Display the map
humidity_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [40]:
# Narrow down cities that fit criteria and drop any results with null values

# Narrow Down To Favorable Temperature Range
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <27) & (city_data_df["Max Temp"] >21)]

# Narrow Daown To Favorable Wind Speed
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] <4.5)]

# Narrow Down Based On Cloudy Conditions (Cloudiness)
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] ==0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,tazacorte,28.6290,-17.9293,23.42,90,0,0.89,ES,1691531786
35,35,vadso,70.0744,29.7487,21.49,88,0,2.06,NO,1691531788
100,100,ribera,37.4984,13.2642,21.47,61,0,1.72,IT,1691531797
112,112,bulanik,39.0929,42.2703,21.88,33,0,1.42,TR,1691531799
174,174,saint-gilles,43.6766,4.4302,22.60,52,0,0.51,FR,1691531807
219,219,hammerfest,70.6634,23.6821,23.25,73,0,1.03,NO,1691531812
309,309,khash,28.2211,61.2158,26.01,21,0,1.45,IR,1691531825
323,323,botou,38.0667,116.5667,23.31,88,0,1.84,CN,1691531697
403,403,cabinda,-5.5500,12.2000,22.08,94,0,2.06,AO,1691531839
426,426,prado,-17.3411,-39.2208,22.46,87,0,3.04,BR,1691531842


### Step 3: Create a new DataFrame called `hotel_df`.

In [41]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
22,tazacorte,ES,28.6290,-17.9293,90,
35,vadso,NO,70.0744,29.7487,88,
100,ribera,IT,37.4984,13.2642,61,
112,bulanik,TR,39.0929,42.2703,33,
174,saint-gilles,FR,43.6766,4.4302,52,
219,hammerfest,NO,70.6634,23.6821,73,
309,khash,IR,28.2211,61.2158,21,
323,botou,CN,38.0667,116.5667,88,
403,cabinda,AO,-5.5500,12.2000,94,
426,prado,BR,-17.3411,-39.2208,87,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories" : "accomodation.hotel", "apiKey" :geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{radius},{longitude},{latitude}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tazacorte - nearest hotel: No hotel found
vadso - nearest hotel: No hotel found
ribera - nearest hotel: No hotel found
bulanik - nearest hotel: No hotel found
saint-gilles - nearest hotel: No hotel found
hammerfest - nearest hotel: No hotel found
khash - nearest hotel: No hotel found
botou - nearest hotel: No hotel found
cabinda - nearest hotel: No hotel found
prado - nearest hotel: No hotel found
nova vicosa - nearest hotel: No hotel found
bokovskaya - nearest hotel: No hotel found
sirte - nearest hotel: No hotel found
lata - nearest hotel: No hotel found
es castell - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
22,tazacorte,ES,28.6290,-17.9293,90,No hotel found
35,vadso,NO,70.0744,29.7487,88,No hotel found
100,ribera,IT,37.4984,13.2642,61,No hotel found
112,bulanik,TR,39.0929,42.2703,33,No hotel found
174,saint-gilles,FR,43.6766,4.4302,52,No hotel found
219,hammerfest,NO,70.6634,23.6821,73,No hotel found
309,khash,IR,28.2211,61.2158,21,No hotel found
323,botou,CN,38.0667,116.5667,88,No hotel found
403,cabinda,AO,-5.5500,12.2000,94,No hotel found
426,prado,BR,-17.3411,-39.2208,87,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height=500,
    size = "Humidity",
    color = "City",
    alpha = 0.8,
    hover_cols = ["Hotel Name", "Country"])

# Display the map
hotel_map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)